In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from pennylane import AngleEmbedding

In [2]:
dev = qml.device("default.qubit", wires=4)

In [3]:
def rot(x):
    qml.Rot(x[0], x[0], x[0], wires=0)
    qml.Rot(x[1], x[1], x[1], wires=1)
    qml.Rot(x[2], x[2], x[2], wires=2)
    qml.Rot(x[3], x[3], x[3], wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])
    return np.array(x)



In [4]:
x = np.array([0.53896774, 0.79503606, 0.27826503, 0.0], requires_grad=False)
ang = rot(x)

print(ang)
@qml.qnode(dev)
def test(angles):

    rot(angles)
    return qml.expval(qml.PauliZ(0))


test(ang)

print("x               : ", x)
print("angles          : ", ang)
#print("amplitude vector: ", np.real(dev.state))

[0.53896774 0.79503606 0.27826503 0.        ]
x               :  [0.53896774 0.79503606 0.27826503 0.        ]
angles          :  [0.53896774 0.79503606 0.27826503 0.        ]


In [5]:
def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)    
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

In [6]:
@qml.qnode(dev)
def circuit(weights, angles):
    rot(angles)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))


def variational_classifier(weights, bias, angles):
    return circuit(weights, angles) + bias


def cost(weights, bias, features, labels):
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)

In [7]:
data = np.loadtxt("variational classifier/iris_classes1and2_scaled.txt")
X = data[:, 0:4]
print("First X sample (original)  :", X[0])

#Quick normalization
X_norm = np.array([x / np.linalg.norm(x) for x in X])
print("First X sample (normalized):", X_norm[0])

# angles for state preparation are new features
features = np.array([rot(x) for x in X_norm], requires_grad=False)
print("First features sample      :", features[0])

Y = data[:, -1]

First X sample (original)  : [0.4  0.75 0.2  0.05]
First X sample (normalized): [0.45732956 0.85749293 0.22866478 0.0571662 ]
First features sample      : [0.45732956 0.85749293 0.22866478 0.0571662 ]


In [8]:
np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))
feats_train = features[index[:num_train]]
Y_train = Y[index[:num_train]]
feats_val = features[index[num_train:]]
Y_val = Y[index[num_train:]]

# We need these later for plotting
X_train = X[index[:num_train]]
X_val = X[index[num_train:]]

In [9]:
num_qubits = 4
num_layers = 6

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

In [10]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [11]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [12]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [13]:
opt = qml.optimize.AdamOptimizer(0.1)
batch_size = 5

# train the variational classifier
weights = weights_init
bias = bias_init
for it in range(60):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val)
    )

Iter:     1 | Cost: 1.6342893 | Acc train: 0.4800000 | Acc validation: 0.5600000 
Iter:     2 | Cost: 1.2132308 | Acc train: 0.4800000 | Acc validation: 0.5600000 
Iter:     3 | Cost: 1.2107145 | Acc train: 0.4800000 | Acc validation: 0.5600000 
Iter:     4 | Cost: 1.2856098 | Acc train: 0.4800000 | Acc validation: 0.5600000 
Iter:     5 | Cost: 1.2051608 | Acc train: 0.4800000 | Acc validation: 0.5600000 
Iter:     6 | Cost: 1.0334591 | Acc train: 0.4800000 | Acc validation: 0.5600000 
Iter:     7 | Cost: 0.8358860 | Acc train: 0.5066667 | Acc validation: 0.5600000 
Iter:     8 | Cost: 0.6623393 | Acc train: 0.9866667 | Acc validation: 1.0000000 
Iter:     9 | Cost: 0.6256633 | Acc train: 0.9600000 | Acc validation: 0.9600000 
Iter:    10 | Cost: 0.5748262 | Acc train: 0.9866667 | Acc validation: 1.0000000 
Iter:    11 | Cost: 0.5179456 | Acc train: 0.9866667 | Acc validation: 1.0000000 
Iter:    12 | Cost: 0.5007940 | Acc train: 1.0000000 | Acc validation: 1.0000000 
Iter:    13 | Co